<a href="https://colab.research.google.com/github/PrithS24/CUET_DuoVation-UNLP2025spanDetection/blob/main/Unlpfinal2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/unlp-workshop/unlp-2025-shared-task.git

Cloning into 'unlp-2025-shared-task'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 81 (delta 5), reused 10 (delta 5), pack-reused 67 (from 1)
Receiving objects: 100% (81/81), 4.59 MiB | 6.45 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [ ]:
%cd unlp-2025-shared-task/data
import os
os.listdir()

/content/unlp-2025-shared-task/data


['techniques-en.md',
 'techniques_classification',
 'span_detection',
 'techniques-uk.md']

In [ ]:
%cd span_detection
os.listdir()

/content/unlp-2025-shared-task/data/span_detection


['test.csv', 'train.parquet', 'solution.csv']

In [ ]:
!pip install transformers datasets pandas scikit-learn torch numpy huggingface_hub emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

df = pd.read_parquet("train.parquet")

# Display the first few rows
df.head()

,id,content,lang,manipulative,techniques,trigger_words
0,0bb0c7fa-101b-4583-a5f9-9d503339141c,Новий огляд мапи DeepState від російського вій...,uk,True,"[euphoria, loaded_language]","[[27, 63], [65, 88], [90, 183], [186, 308]]"
1,7159f802-6f99-4e9d-97bd-6f565a4a0fae,Недавно 95 квартал жёстко поглумился над русск...,ru,True,"[loaded_language, cherry_picking]","[[0, 40], [123, 137], [180, 251], [253, 274]]"
2,e6a427f1-211f-405f-bd8b-70798458d656,🤩\nТим часом йде евакуація Бєлгородського авто...,uk,True,"[loaded_language, euphoria]","[[55, 100]]"
3,1647a352-4cd3-40f6-bfa1-d87d42e34eea,В Україні найближчим часом мають намір посилит...,uk,False,None,None
4,9c01de00-841f-4b50-9407-104e9ffb03bf,"Расчёты 122-мм САУ 2С1 ""Гвоздика"" 132-й бригад...",ru,True,[loaded_language],"[[114, 144]]"


In [ ]:
# Step 1: Ensure missing values are handled properly
df['techniques'] = df['techniques'].apply(lambda x: [] if (x is None or isinstance(x, float)) else x)
df['trigger_words'] = df['trigger_words'].apply(lambda x: [] if (x is None or isinstance(x, float)) else x)

# Step 2: Verify that no missing values remain
print(df.isnull().sum())  # Should print all 0s

# Check if all techniques and trigger_words are lists
print(df.head(10))


id               0
content          0
lang             0
manipulative     0
techniques       0
trigger_words    0
dtype: int64
                                     id  \
0  0bb0c7fa-101b-4583-a5f9-9d503339141c   
1  7159f802-6f99-4e9d-97bd-6f565a4a0fae   
2  e6a427f1-211f-405f-bd8b-70798458d656   
3  1647a352-4cd3-40f6-bfa1-d87d42e34eea   
4  9c01de00-841f-4b50-9407-104e9ffb03bf   
5  46493f44-f00a-4ffb-9cda-252ccf5fa4c6   
6  1417fffa-461c-4bf7-bea5-398e8ea5db81   
7  f8021666-1636-4e1f-b8e3-535860eca5ce   
8  ed5d2195-09b4-4837-82eb-b65244c8a7b2   
9  7d7504ff-295e-435d-b565-ed62d4eebaa0   

                                             content lang  manipulative  \
0  Новий огляд мапи DeepState від російського вій...   uk          True   
1  Недавно 95 квартал жёстко поглумился над русск...   ru          True   
2  🤩\nТим часом йде евакуація Бєлгородського авто...   uk          True   
3  В Україні найближчим часом мають намір посилит...   uk         False   
4  Расчёты 122-мм САУ 2С

In [ ]:
import re
# Convert all text to lowercase while keeping structure intact
df["content"] = df["content"].str.lower()

def clean_text(text):
    """Clean text without breaking token positions."""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"\@\w+", "", text)  # Remove mentions (@username)
    return text

df["content"] = df["content"].apply(clean_text)


In [ ]:
ukrainian_emoji_dict = {
    # ⚡ Urgency & Breaking News
    "⚡": "терміново",  # "urgent" / "breaking news"
    "⚠️": "попередження",  # "warning"
    "🚨": "сирена",  # "alert"
    "❗": "важливо",  # "important"
    "❓": "питання",  # "questionable"

    # 🎭 Emotional Manipulation
    "😭": "плач",  # "crying" (used to exaggerate sadness)
    "😢": "смуток",  # "sadness"
    "😂": "сміх",  # "laughter" (mockery in propaganda)
    "😡": "гнів",  # "anger" (provoking emotions)
    "🤬": "обурення",  # "rage" (hate speech & provocation)
    "😱": "паніка",  # "panic" (fear appeal)
    "😨": "переляк",  # "fear"

    # 🔥 Propaganda & Exaggeration
    "🔥": "сенсація",  # "sensationalism" (exaggeration)
    "💥": "вибухова новина",  # "explosive news"
    "💣": "загроза",  # "threat" (fear-based manipulation)
    "⚔️": "війна",  # "war"

    # ❤️ Patriotism & Unity Appeals
    "🇺🇦": "україна",  # "Ukraine" (nationalism & unity)
    "🇷🇺": "росія",  # "Russia" (propaganda context)
    "❤️": "любов",  # "love" (nationalistic appeals)
    "🤝": "єдність",  # "unity"
    "💪": "сила",  # "strength" (mobilization)

    # 🎯 Call to Action & Persuasion
    "🚀": "перемога",  # "victory" (used in war propaganda)
    "🏆": "успіх",  # "success"
    "💰": "корупція",  # "corruption" (often used in political posts)

    # 🏛️ Government & Authority
    "🏛️": "влада",  # "government"
    "📢": "пропаганда",  # "propaganda"
    "📜": "закон",  # "law"

    # 📡 Media & Information Manipulation
    "📢": "фейк",  # "fake news"
    "📺": "телебачення",  # "media influence"
    "📰": "змі",  # "mass media"
    "💻": "кібервійна",  # "cyberwar"

    # 🎭 Distrust & Confusion
    "🤔": "сумнів",  # "doubt"
    "🙄": "скептицизм",  # "skepticism"
    "🧐": "маніпуляція",  # "manipulation"

    # **Restoring Full Set of Emojis Below**

    # People & Actions
    "👨": "чоловік",
    "👩": "жінка",
    "👮": "поліцейський",
    "👷": "будівельник",
    "🧑‍⚕️": "лікар",
    "🧑‍🎓": "студент",
    "🧑‍🏫": "вчитель",

    # Love & Affection
    "😍": "захоплення любов",  # Heart eyes emoji
    "🥰": "кохання ніжність",  # Smiling with hearts emoji
    "🤩": "зіркові очі",  # "star eyes" (amazement, awe)
    "😘": "поцілунок",
    "😗": "поцілунок",
    "😚": "поцілунок",
    "😙": "поцілунок",
    "💙": "синє серце",
    "💛": "жовте серце",
    "💜": "фіолетове серце",
    "💔": "розбите серце",
    "💕": "кохання",
    "💖": "захоплення",
    "💘": "закоханість",
    "😊": "посмішка",  # "smiling face with smiling eyes" (happiness)
    "😉": "підморгування",  # "winking face" (playfulness)
    "😆": "сміх до сліз",  # "laughing face with tightly closed eyes"
    "😄": "щасливе обличчя",  # "grinning face with smiling eyes"
    "😬": "криве обличчя",  # "grimacing face" (nervousness)
    "😋": "смачно",  # "face savoring food"
    "🤤": "смак",  # "drooling face" (delicious, food-related)
    "😎": "крутість",  # "smiling face with sunglasses" (cool)
    "😔": "сумний",  # "pensive face"
    "😞": "невдоволення",  # "disappointed face"
    "😒": "невдоволення",  # "unamused face"
    "🤧": "чихання",  # "sneezing face" (illness, sickness)

    # Gestures
    "👍": "схвалення",
    "👎": "несхвалення",
    "✊": "кулак",
    "🤛": "лівий кулак",
    "🤜": "правий кулак",
    "👏": "аплодисменти",
    "👐": "розкриті руки",
    "🤲": "молитва",
    "🤝": "рукостискання",
    "👉": "вказівний палець",  # "pointing finger"
    "👈": "вказівний палець лівий",  # "left pointing finger"
    "👌": "окей",  # "ok" (thumb and index forming a circle)
    "✋": "стоп",  # "stop" (raised hand)
    "👋": "привітання",  # "waving hand"
    "🖕": "середній палець",  # "middle finger" (disrespect)
    "🤞": "палець схрещений",  # "fingers crossed" (wish for good luck)
    "🙏": "молитва",

    # Fire, Explosion, War
    "🏹": "лук",
    "🛡️": "щит",

    # Money & Economy
    "💳": "кредитна картка",
    "🏦": "банк",

    # Nature & Weather
    "☀️": "сонце",
    "🌞": "сонячний день",
    "🌧️": "дощ",
    "⛈️": "гроза",
    "❄️": "сніг",
    "🌊": "хвиля",

    # Animals
    "🐶": "собака",
    "🐱": "кішка",
    "🦁": "лев",
    "🐺": "вовк",
    "🦊": "лисиця",
    "🐴": "кінь",
    "🐻": "ведмідь",

    # Food & Drinks
    "🍞": "хліб",
    "🥖": "батон",
    "🥔": "картопля",
    "🥕": "морква",
    "🍎": "яблуко",
    "🍇": "виноград",
    "🍓": "полуниця",
    "🥩": "м'ясо",
    "🍗": "курка",
    "🍻": "пиво",
    "🥂": "шампанське",

    # Transport
    "🚗": "автомобіль",
    "🚕": "таксі",
    "🚙": "джип",
    "🚌": "автобус",
    "🚆": "поїзд",
    "✈️": "літак",

    # Technology & Media
    "📱": "телефон",
    "💻": "ноутбук",
    "🎥": "кіно",
    "📸": "камера",
}

import emoji

def replace_emojis(text, lang):
    if lang == 'ru':
        # Use emoji.demojize with Russian descriptions
        text_with_desc = emoji.demojize(text, language="ru")
        return text_with_desc.replace(":", "").replace("_", " ")

    elif lang == 'uk':
        # Manually replace emojis with Ukrainian words
        for emoji_symbol, replacement in ukrainian_emoji_dict.items():
            text = text.replace(emoji_symbol, replacement)
        return text  # Return modified Ukrainian text

    return text  # Return unchanged text if language is neither RU nor UK

df['content'] = df.apply(lambda x: replace_emojis(x['content'], x['lang']), axis=1)


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import f1_score
from datasets import Dataset
from transformers import EarlyStoppingCallback

In [ ]:
# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.cross_entropy = nn.CrossEntropyLoss(ignore_index=-100, reduction='none')

    def forward(self, logits, labels):
        ce_loss = self.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1))
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

In [ ]:
# Use XLM-RoBERTa model directly for token classification
class XLMRobertaWithFocalLoss(nn.Module):
    def __init__(self, model_name='xlm-roberta-base', num_labels=2, dropout=0.2):
        super(XLMRobertaWithFocalLoss, self).__init__()
        self.xlm_roberta = XLMRobertaForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
        self.dropout = nn.Dropout(dropout)
        self.focal_loss = FocalLoss(gamma=2, alpha=0.25)

    def forward(self, input_ids, attention_mask, labels=None):
        # Forward pass through XLM-RoBERTa
        outputs = self.xlm_roberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Shape: (batch_size, seq_len, num_labels)

        loss = None
        if labels is not None:
            loss = self.focal_loss(logits, labels)

        return {'loss': loss, 'logits': logits}

In [ ]:
# Load the tokenizer
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
# Tokenize the dataset
# Tokenize and prepare dataset
def label_spans(row, tokenizer, max_length=512):
    encoding = tokenizer(
        row['content'], truncation=True, padding='max_length',
        max_length=max_length, return_offsets_mapping=True,
        return_attention_mask=True
    )
    labels = [0] * max_length
    if row['trigger_words'] is not None and len(row['trigger_words']) > 0:
        for span in row['trigger_words']:
            start, end = span
            start_token = encoding.char_to_token(start)
            end_token = encoding.char_to_token(end - 1)
            if start_token is not None and end_token is not None:
                for idx in range(start_token, end_token + 1):
                    labels[idx] = 1
    return encoding.input_ids, encoding.attention_mask, labels

# Apply tokenization to the dataset
tokenized_data = df.apply(lambda row: label_spans(row, tokenizer), axis=1)
dataset = Dataset.from_dict({
    'input_ids': [x[0] for x in tokenized_data],
    'attention_mask': [x[1] for x in tokenized_data],
    'labels': [x[2] for x in tokenized_data]
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [ ]:
# Split into train and validation datasets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']


In [ ]:
#  Define evaluation metric (F1 score)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1).flatten()
    labels = labels.flatten()
    mask = labels != -100
    labels = labels[mask]
    predictions = predictions[mask]
    f1 = f1_score(labels, predictions, average='binary', pos_label=1)
    return {'f1': f1}


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Evaluate every epoch
    save_strategy='epoch',  # Save at the end of every epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,  # Increase epochs
    logging_dir='./logs',
    logging_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    evaluation_strategy="epoch",
    save_steps=500,
    report_to='none',
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Instantiate the model
model = XLMRobertaWithFocalLoss(
    model_name='xlm-roberta-base',
    num_labels=2,
    dropout=0.2
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping after 3 epochs without improvement
)

# Step 6: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.008329,0.307305
2,No log,0.008072,0.487864
3,0.009800,0.008216,0.484325
4,0.009800,0.008589,0.381331
5,0.009800,0.008724,0.509552
6,0.006700,0.009479,0.531558
7,0.006700,0.010771,0.487586
8,0.004700,0.011100,0.473135
9,0.004700,0.012973,0.458426


TrainOutput(global_step=1728, training_loss=0.00659438043280884, metrics={'train_runtime': 3532.3921, 'train_samples_per_second': 12.981, 'train_steps_per_second': 0.811, 'total_flos': 0.0, 'train_loss': 0.00659438043280884, 'epoch': 9.0})

In [ ]:
# Step 8: Evaluate and display final F1 score
logs = trainer.state.log_history
final_f1_score = None
for log in reversed(logs):
    if 'eval_f1' in log:
        final_f1_score = log['eval_f1']
        break

if final_f1_score is not None:
    f1_percentage = final_f1_score * 100
    print(f"Final F1 Score: {f1_percentage:.2f}%")
else:
    print("F1 score not found in logs.")


Final F1 Score: 45.84%


In [ ]:
# Step 7: Threshold Tuning on Validation Set
def get_best_threshold(model, dataset, thresholds=np.arange(0.1, 0.9, 0.05)):
    model.eval().cuda()
    preds, true_labels = [], []

    for batch in dataset:
        input_ids = torch.tensor(batch['input_ids']).unsqueeze(0).cuda()
        attention_mask = torch.tensor(batch['attention_mask']).unsqueeze(0).cuda()
        labels = np.array(batch['labels'])

        with torch.no_grad():
            logits = model(input_ids, attention_mask)['logits']
            probs = torch.softmax(logits, dim=-1)[0, :, 1].cpu().numpy()

        preds.append(probs)
        true_labels.append(labels)

    preds = np.concatenate(preds)
    true_labels = np.concatenate(true_labels)

    best_f1, best_threshold = 0, 0.3
    for thresh in thresholds:
        pred_labels = (preds > thresh).astype(int)
        f1 = f1_score(true_labels, pred_labels, average='macro')
        if f1 > best_f1:
            best_f1, best_threshold = f1, thresh

    print(f"Optimal Threshold: {best_threshold:.2f}, F1 Score: {best_f1:.4f}")
    return best_threshold

In [ ]:
# Find best threshold
optimal_threshold = get_best_threshold(model, val_dataset)

Optimal Threshold: 0.40, F1 Score: 0.7566


In [ ]:
# Step 9: Save after training is complete
torch.save(model.state_dict(), './results/model.pt')
tokenizer.save_pretrained('./results/tokenizer')

('./results/tokenizer/tokenizer_config.json',
 './results/tokenizer/special_tokens_map.json',
 './results/tokenizer/sentencepiece.bpe.model',
 './results/tokenizer/added_tokens.json',
 './results/tokenizer/tokenizer.json')

In [ ]:
# Step 10: Prepare test data and make predictions
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')

# # Path to the test.csv file in Google Drive
# file_path = '/content/drive/MyDrive/Colab Notebooks/test (3).csv'  # Modify this path according to where your file is located
import pandas as pd

# Preprocess the content of the test data
test_df = pd.read_csv("test.csv")
test_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,content
0,521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba,"Они просрали нашу технику, положили кучу людей..."
1,9b2a61e4-d14e-4ff7-b304-e73d720319bf,❗️\nКитай предлагает отдать оккупированные тер...
2,f0f1c236-80a8-4d25-b30c-a420a39be632,Сегодня будет ровно 6 месяцев с этого обещания...
3,31ea05ba-2c2b-4b84-aba7-f3cf6841b204,⚡️\nІзраїль вперше у світі збив балістичну рак...
4,a79e13ec-6d9a-40b5-b54c-7f4f743a7525,Склав невелику навчально-методичну таблицю на ...


In [ ]:
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"\@\w+", "", text)
    text = text.lower()
    return text

test_df['content'] = test_df['content'].apply(clean_text)

In [ ]:
# Tokenize Test Data
def prepare_test_data(test_df, tokenizer, max_length=512):
    encodings = []
    for content in test_df['content']:
        encodings.append(tokenizer(content, truncation=True,
                                   padding='max_length',
                                   max_length=max_length,
                                   return_offsets_mapping=True,
                                   return_tensors='pt'))
    return encodings

test_encodings = prepare_test_data(test_df, tokenizer)


In [ ]:
#Predict function with optimal threshold
def predict(test_encodings, model, threshold):
    predictions = []
    model.eval().cuda()
    for i, encoding in enumerate(test_encodings):
        input_ids = encoding['input_ids'].cuda()
        attention_mask = encoding['attention_mask'].cuda()
        with torch.no_grad():
            logits = model(input_ids, attention_mask)['logits']
        probs = torch.softmax(logits, dim=-1)[0, :, 1]
        pred_labels = (probs > threshold).int().cpu().numpy()
        offsets = encoding['offset_mapping'][0].numpy()

        spans, current_span = [], None
        for idx, label in enumerate(pred_labels):
            if label == 1:
                if current_span is None:
                    current_span = [offsets[idx][0], offsets[idx][1]]
                else:
                    current_span[1] = offsets[idx][1]
            else:
                if current_span:
                    spans.append(tuple(current_span))
                    current_span = None
        if current_span:
            spans.append(tuple(current_span))
        predictions.append({'id': test_df.iloc[i]['id'], 'trigger_words': spans})
    return predictions

# Predict and save using optimal threshold
preds = predict(test_encodings, model, optimal_threshold)
pred_df = pd.DataFrame(preds)

In [ ]:
# Modify this line before saving the predictions to CSV
print(pred_df.head())
pred_df['trigger_words'] = pred_df['trigger_words'].apply(
    lambda spans: [(int(start), int(end)) for start, end in spans]
)
pred_df.to_csv('/content/drive/MyDrive/Colab Notebooks/submission3.csv', index=False)

print("✅ Predictions saved to submission.csv with optimal threshold!")

                                     id                     trigger_words
0  521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba                        [(0, 253)]
1  9b2a61e4-d14e-4ff7-b304-e73d720319bf  [(20, 26), (56, 68), (374, 428)]
2  f0f1c236-80a8-4d25-b30c-a420a39be632                        [(0, 127)]
3  31ea05ba-2c2b-4b84-aba7-f3cf6841b204                                []
4  a79e13ec-6d9a-40b5-b54c-7f4f743a7525           [(87, 105), (127, 309)]
✅ Predictions saved to submission.csv with optimal threshold!


In [ ]:
import pandas as pd
import pandas.api.types
from sklearn.metrics import f1_score
import ast


class ParticipantVisibleError(Exception):
    """Custom exception for participant-visible errors."""
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Compute span-level F1 score based on overlap.

    Parameters:
    - solution (pd.DataFrame): Ground truth DataFrame with row ID and token labels.
    - submission (pd.DataFrame): Submission DataFrame with row ID and token labels.
    - row_id_column_name (str): Column name for the row identifier.

    Returns:
    - float: The token-level weighted F1 score.

    Example:
    >>> solution = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (725, 831)], [(300, 312)], []]
    ... })
    >>> submission = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (700, 720)], [(300, 312)], [(100, 200)]]
    ... })
    >>> score(solution, submission, "id")
    0.16296296296296295
    """
    if not all(col in solution.columns for col in ["id", "trigger_words"]):
        raise ValueError("Solution DataFrame must contain 'id' and 'trigger_words' columns.")
    if not all(col in submission.columns for col in ["id", "trigger_words"]):
        raise ValueError("Submission DataFrame must contain 'id' and 'trigger_words' columns.")

    def safe_parse_spans(trigger_words):
        if isinstance(trigger_words, str):
            try:
                return ast.literal_eval(trigger_words)
            except (ValueError, SyntaxError):
                return []
        if isinstance(trigger_words, (list, tuple)):
            return trigger_words
        return []

    def extract_tokens_from_spans(spans):
        tokens = set()
        for start, end in spans:
            tokens.update(range(start, end))
        return tokens

    solution = solution.copy()
    submission = submission.copy()

    solution["trigger_words"] = solution["trigger_words"].apply(safe_parse_spans)
    submission["trigger_words"] = submission["trigger_words"].apply(safe_parse_spans)

    merged = pd.merge(
        solution,
        submission,
        on="id",
        suffixes=("_solution", "_submission")
    )

    total_true_tokens = 0
    total_pred_tokens = 0
    overlapping_tokens = 0

    for _, row in merged.iterrows():
        true_spans = row["trigger_words_solution"]
        pred_spans = row["trigger_words_submission"]

        true_tokens = extract_tokens_from_spans(true_spans)
        pred_tokens = extract_tokens_from_spans(pred_spans)

        total_true_tokens += len(true_tokens)
        total_pred_tokens += len(pred_tokens)
        overlapping_tokens += len(true_tokens & pred_tokens)

    precision = overlapping_tokens / total_pred_tokens if total_pred_tokens > 0 else 0
    recall = overlapping_tokens / total_true_tokens if total_true_tokens > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1

In [ ]:
solution = pd.read_csv('solution.csv')
# Prepare submission DataFrame (predictions from the model)
submission = pred_df.copy()  # Assuming pred_df is your model's output
submission["trigger_words"] = submission["trigger_words"].apply(lambda x: [(int(start), int(end)) for start, end in x])  # Convert span format

# Now, call the `score` function to get the F1 score:
span_level_f1 = score(solution, submission, "id")

print(f"Span-level F1 score: {span_level_f1:.4f}")

Span-level F1 score: 0.5821
